#  Writers Room: UI Feedback Loop for Series Overview

In [ ]:
import json
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

load_dotenv()
client = OpenAI()
MEMORY_FILE = Path("../memory/writers_room_memory.json")

In [2]:
def init_memory():
    MEMORY_FILE.parent.mkdir(parents=True, exist_ok=True)
    if not MEMORY_FILE.exists():
        with open(MEMORY_FILE, "w") as f:
            json.dump({}, f)

def load_memory():
    init_memory()
    with open(MEMORY_FILE, "r") as f:
        return json.load(f)

def save_to_memory(table: str, key: str, content):
    init_memory()
    with open(MEMORY_FILE, "r+") as f:
        db = json.load(f)
        db.setdefault(table, {})[key] = content
        f.seek(0)
        json.dump(db, f, indent=2)
        f.truncate()

In [3]:
def generate_series_overview(thesis):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a showrunner developing a series overview."},
            {"role": "user", "content": f"Generate a series overview for: {thesis}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

def revise_series_overview(previous_output, feedback):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a showrunner revising a series overview."},
            {"role": "user", "content": f"Revise this series overview:\n\n{previous_output}\n\nBased on this feedback:\n{feedback}"}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

In [10]:
def ui_feedback_loop(title, generate_func, revise_func, *args):
    versions = []
    feedback_history = []

    # Initial output
    current_output = generate_func(*args)
    versions.append(current_output)

    output_display = widgets.Output()
    feedback_input = widgets.Textarea(placeholder='Type feedback or leave blank to finish')
    revise_button = widgets.Button(description='Revise', button_style='primary')
    complete_button = widgets.Button(description=f'Save Current Version', button_style='success')

    def show_latest():
        with output_display:
            clear_output()
            display(Markdown(f"### {title} (Version {len(versions)})\n\n{versions[-1]}"))

    def on_revise_click(b):
        feedback = feedback_input.value.strip()
        if feedback:
            feedback_history.append(feedback)
            new_output = revise_func(versions[-1], feedback)
            versions.append(new_output)
            feedback_input.value = ''
            show_latest()

    def on_accept_click(b):
        final_version = versions[-1]
        save_to_memory("series_plan", title.lower().replace(' ', '_'), {
            "final_version": final_version,
            "feedback_history": feedback_history
        })
        with output_display:
            display(Markdown(f"✅ **Saved Version {len(versions)} to memory under 'series_plan' → '{title.lower().replace(' ', '_')}'**"))

    revise_button.on_click(on_revise_click)
    complete_button.on_click(on_accept_click)

    show_latest()
    display(widgets.VBox([
        output_display,
        feedback_input,
        widgets.HBox([revise_button, complete_button])
    ]))

In [ ]:
# Load thesis and run
memory = load_memory()
thesis = memory.get("concept", {}).get("show_thesis", {}).get("final_thesis", "")
if not thesis:
    raise ValueError("❌ No final show thesis found. Please generate it first.")
ui_feedback_loop("Series Overview", generate_series_overview, revise_series_overview, thesis)